# Alternative ways to execute runnables
* Invoke, Stream and Batch.

## Setup

#### After you download the code from the github repository in your computer
In terminal:
* cd project_name
* pyenv local 3.11.4
* poetry install
* poetry shell

#### To open the notebook with Jupyter Notebooks
In terminal:
* jupyter lab

Go to the folder of notebooks and open the right notebook.

#### To see the code in Virtual Studio Code or your editor of choice.
* open Virtual Studio Code or your editor of choice.
* open the project-folder
* open the 004-invoke-stream-batch.py file

## Create your .env file
* In the github repo we have included a file named .env.example
* Rename that file to .env file and here is where you will add your confidential api keys. Remember to include:
* OPENAI_API_KEY=your_openai_api_key
* LANGCHAIN_TRACING_V2=true
* LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
* LANGCHAIN_API_KEY=your_langchain_api_key
* LANGCHAIN_PROJECT=your_project_name

We will call our LangSmith project **004-invoke-stream-batch**.

## Connect with the .env file located in the same directory of this notebook

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [1]:
#!pip install python-dotenv

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
google_api_key = os.environ["GOOGLE_API_KEY"]
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=google_api_key)

#### Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [3]:
#!pip install langchain

## Connect with an LLM

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [4]:
#!pip install langchain-openai

* NOTE: Since right now is the best LLM in the market, we will use OpenAI by default. You will see how to connect with other Open Source LLMs like Llama3 or Mistral in a next lesson.

In [5]:
# from langchain_openai import ChatOpenAI

# model = ChatOpenAI(model="gpt-3.5-turbo-0125")

## LCEL Chain
* **An LCEL Chain is a Sequence of Runnables.**
* Almost any component in LangChain (prompts, models, output parsers, vector store retrievers, tools, etc) can be used as a Runnable.
* Runnables can be chained together using the pipe operator `|`. The resulting chains of runnables are also runnables themselves.
* The order (left to right) of the elements in a LCEL chain matters.

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [7]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player} but short and basic answer")

output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"soccer_player": "Ronaldo"})

'Cristiano Ronaldo is the only player to score 50+ goals in six consecutive seasons.'

* All the components of the chain are Runnables.
* **When we write chain.invoke() we are using invoke with all the componentes of the chain in an orderly manner**:
    * First, we apply .invoke() with the user input to the prompt template.
    * Then, with the completed prompt, we apply .invoke() to the model.
    * And finally, we apply .invoke() to the output parser with the output of the model.
* IMPORTANT: the order of operations in a chain matters. If you try to execute the previous chain with the components in different order, the chain will fail.

## Let's see this graphically

![alt text](./lcel-2.png)


## And now let's replicate this process without using the LCEL chain

#### First, we apply .invoke() with the user input to the prompt template.

In [8]:
prompt.invoke({"soccer_player": "Ronaldo"})

ChatPromptValue(messages=[HumanMessage(content='tell me a curious fact about Ronaldo but short and basic answer', additional_kwargs={}, response_metadata={})])

#### Then, with the completed prompt, we apply .invoke() to the model.

In [9]:
from langchain_core.messages.human import HumanMessage

output_after_first_step = [HumanMessage(content='tell me a curious fact about Ronaldo')]

In [10]:
model.invoke(output_after_first_step)

AIMessage(content="Here's a curious fact about Cristiano Ronaldo:\n\nDespite being known for his incredible jumping ability and powerful headers, Cristiano Ronaldo actually holds a record for the highest recorded jump by a soccer player to score a goal. He jumped 2.56 meters (8 feet 5 inches) in the air to score a header against Sampdoria in 2019 while playing for Juventus. What makes it even more impressive is that his head made contact with the ball at a height of 71 centimeters (28 inches) off the ground.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--c849e707-018b-4bba-8c25-94543353da9a-0', usage_metadata={'input_tokens': 7, 'output_tokens': 112, 'total_tokens': 119, 'input_token_details': {'cache_read': 0}})

#### And finally, we apply .invoke() to the output parser with the output of the model.

In [11]:
from langchain_core.messages.ai import AIMessage

output_after_second_step = AIMessage(content='One curious fact about Cristiano Ronaldo is that he does not have any tattoos on his body. Despite the fact that many professional athletes have tattoos, Ronaldo has chosen to keep his body ink-free.', response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 14, 'total_tokens': 52}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c9812511-043a-458a-bfb8-005bc0d057fb-0', usage_metadata={'input_tokens': 14, 'output_tokens': 38, 'total_tokens': 52})

In [12]:
output_parser.invoke(output_after_second_step)

'One curious fact about Cristiano Ronaldo is that he does not have any tattoos on his body. Despite the fact that many professional athletes have tattoos, Ronaldo has chosen to keep his body ink-free.'

## Ways to execute Runnables
* Remember:
    * An LCEL Chain is a Sequence of Runnables.
    * Almost any component in LangChain (prompts, models, output parsers, etc) can be used as a Runnable.
    * Runnables can be chained together using the pipe operator `|`. The resulting chains of runnables are also runnables themselves.
    * The order (left to right) of the elements in a LCEL chain matters.

#### LCEL Chains/Runnables are used with:
* chain.invoke(): call the chain on an input.
* chain.stream(): call the chain on an input and stream back chunks of the response.
* chain.batch(): call the chain on a list of inputs.

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("Tell me one sentence about {politician}.")
chain = prompt | model

In [14]:
chain.invoke({"politician": "Churchill"})

AIMessage(content='Winston Churchill was a British statesman who served as Prime Minister of the United Kingdom during World War II.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--3621a2d3-7751-4070-be2a-313e0b5b665a-0', usage_metadata={'input_tokens': 7, 'output_tokens': 21, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}})

In [15]:
for s in chain.stream({"politician": "F.D. Roosevelt"}):
    print(s.content, end="", flush=True)

Franklin Delano Roosevelt led the United States through the Great Depression and most of World War II, implementing the New Deal to address economic hardship and guiding the nation as a major Allied power.


#### The for loop explained in simple terms
This `for` loop is used to show responses piece by piece as they are received. Here's how it works in simple terms:

1. **Getting Responses in Parts**: The loop receives pieces of a response from the system one at a time. In this example, the system is responding with information about the politician "F.D. Roosevelt."

2. **Printing Out Responses Immediately**: Each time a new piece of the response is received, it immediately prints it out. The setup makes sure there are no new lines between parts, so it all looks like one continuous response.

3. **No Waiting**: By using this loop, you don't have to wait for the entire response to be ready before you start seeing parts of it. This makes it feel quicker and more like a conversation.

This way, the loop helps provide a smoother and more interactive way of displaying responses from the system as they are generated.

#### The for loop explained in technical terms
This `for` loop is used to handle streaming output from a language model response. Here’s a breakdown of its functionality and context:

1. **Iteration through Streamed Output**: The `for` loop iterates over the generator returned by `chain.stream(...)`. The `stream` method of the `chain` object (which is a combination of a prompt template and a language model) is designed to yield responses incrementally. This is particularly useful when responses from a model are long or need to be displayed in real-time.

2. **Data Fetching**: In this loop, `s` represents each piece of content that is streamed back from the model as the response is being generated. The model in this case is set up to respond with information about the politician "F.D. Roosevelt".

3. **Output Handling**: Inside the loop, `print(s.content, end="", flush=True)` is called for each piece of streamed content. The `print` function is customized with:
   - `end=""`: This parameter ensures that each piece of content is printed without adding a new line after each one, thus allowing the response to appear continuous on a single line.
   - `flush=True`: This parameter forces the output buffer to be flushed immediately after each print statement, ensuring that each piece of content is displayed to the user as soon as it is received without any delay.

By using this loop, the code is able to display each segment of the model's response as it becomes available, making the user interface more dynamic and responsive, especially for real-time applications where prompt feedback is beneficial.

In [16]:
chain.batch([{"politician": "Lenin"}, {"politician": "Stalin"}])

[AIMessage(content='Vladimir Lenin was a Russian revolutionary, politician, and political theorist who served as the first and founding head of government of Soviet Russia.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--ad09077f-2667-4ee4-a163-e9751f246212-0', usage_metadata={'input_tokens': 7, 'output_tokens': 27, 'total_tokens': 34, 'input_token_details': {'cache_read': 0}}),
 AIMessage(content='Joseph Stalin was the dictator of the Soviet Union from the mid-1920s until his death in 1953, known for his totalitarian rule and policies that led to widespread famine and political repression.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--0cbd6a70-9faa-497e-93b6-6a9c762b82e9-0', usage_metadata={'

#### LCEL Chains/Runnables can be also used asynchronously:
* chain.ainvoke(): call the chain on an input.
* chain.astream(): call the chain on an input and stream back chunks of the response.
* chain.abatch(): call the chain on a list of inputs.

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 004-invoke-stream-batch.py
* In terminal, make sure you are in the directory of the file and run:
    * python 004-invoke-stream-batch.py